## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
%matplotlib inline

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esperance,AU,-33.8667,121.9000,58.84,73,0,11.34,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,78.12,73,100,12.66,overcast clouds
2,2,Fortuna,US,40.5982,-124.1573,44.94,58,75,2.10,broken clouds
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,-34.60,67,40,5.75,scattered clouds
4,4,Cayenne,GF,4.9333,-52.3333,82.44,85,75,13.80,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 25
What is your desired maximum temperature for your trip? 45


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Fortuna,US,40.5982,-124.1573,44.94,58,75,2.10,broken clouds
14,14,Albany,US,42.6001,-73.9662,26.55,57,100,1.01,overcast clouds
21,21,Ushuaia,AR,-54.8000,-68.3000,39.65,91,100,30.62,light rain
22,22,Grindavik,IS,63.8424,-22.4338,28.87,93,75,6.91,light snow
28,28,Bluff,NZ,-46.6000,168.3333,42.80,89,10,6.58,clear sky
32,32,Lasa,IT,46.6166,10.7002,31.46,61,0,2.13,clear sky
33,33,Baoning,CN,25.3378,110.0260,40.86,100,75,2.24,mist
35,35,Mataura,NZ,-46.1927,168.8643,41.49,99,29,2.39,scattered clouds
38,38,Korla,CN,41.7597,86.1469,29.73,45,0,4.72,clear sky
51,51,Sivac,RS,45.7033,19.3822,37.35,71,7,10.25,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Fortuna,US,40.5982,-124.1573,44.94,58,75,2.10,broken clouds
14,14,Albany,US,42.6001,-73.9662,26.55,57,100,1.01,overcast clouds
21,21,Ushuaia,AR,-54.8000,-68.3000,39.65,91,100,30.62,light rain
22,22,Grindavik,IS,63.8424,-22.4338,28.87,93,75,6.91,light snow
28,28,Bluff,NZ,-46.6000,168.3333,42.80,89,10,6.58,clear sky
...,...,...,...,...,...,...,...,...,...,...
638,638,Narva-Joesuu,EE,59.4589,28.0408,32.22,92,100,17.07,light snow
650,650,Josanicka Banja,RS,43.3895,20.7503,36.70,99,100,2.77,overcast clouds
675,675,Quesnel,CA,52.9999,-122.5029,34.30,64,100,6.91,overcast clouds
676,676,Vestmanna,FO,62.1564,-7.1664,32.92,69,100,16.11,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Fortuna,US,44.94,broken clouds,40.5982,-124.1573,
14,Albany,US,26.55,overcast clouds,42.6001,-73.9662,
21,Ushuaia,AR,39.65,light rain,-54.8000,-68.3000,
22,Grindavik,IS,28.87,light snow,63.8424,-22.4338,
28,Bluff,NZ,42.80,clear sky,-46.6000,168.3333,
32,Lasa,IT,31.46,clear sky,46.6166,10.7002,
33,Baoning,CN,40.86,mist,25.3378,110.0260,
35,Mataura,NZ,41.49,scattered clouds,-46.1927,168.8643,
38,Korla,CN,29.73,clear sky,41.7597,86.1469,
51,Sivac,RS,37.35,clear sky,45.7033,19.3822,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")   
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Fortuna,US,44.94,broken clouds,40.5982,-124.1573,Super 8 by Wyndham Fortuna
21,Ushuaia,AR,39.65,light rain,-54.8000,-68.3000,Albatross Hotel
22,Grindavik,IS,28.87,light snow,63.8424,-22.4338,Northern Light Inn & Max's Restaurant
28,Bluff,NZ,42.80,clear sky,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
32,Lasa,IT,31.46,clear sky,46.6166,10.7002,Pension Feldgärtenhof


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))